In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌊 Flood Event Detection & Analysis\n",
    "### Identifying and characterizing historical flood events\n",
    "\n",
    "This notebook covers:\n",
    "- Automated flood event detection\n",
    "- Severity classification\n",
    "- Seasonal risk profiling\n",
    "- Year-over-year comparisons"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from src.config import Config\n",
    "from src.data_loader import ERA5DataLoader\n",
    "from src.analytics import FloodAnalytics\n",
    "\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "sns.set_palette('husl')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load processed data\n",
    "station_key = 'koshi_chatara'\n",
    "station_config = Config.TARGET_STATIONS[station_key]\n",
    "\n",
    "loader = ERA5DataLoader(station_config, years_back=5)\n",
    "data = loader.fetch_era5_data()\n",
    "\n",
    "# Initialize analytics\n",
    "analytics = FloodAnalytics(data, station_config)\n",
    "\n",
    "print(f\"Analyzing: {station_config['name']}\")\n",
    "print(f\"Flood threshold: {station_config['flood_stage']:.2f} m\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Detect Flood Events"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Identify all flood events\n",
    "flood_events = analytics.identify_flood_events()\n",
    "\n",
    "print(f\"\\nDetected {len(flood_events)} flood events over {data['year'].nunique()} years\")\n",
    "print(f\"Average: {len(flood_events)/data['year'].nunique():.1f} floods per year\\n\")\n",
    "\n",
    "# Display recent events\n",
    "flood_events.tail(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Severity Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Count by severity\n",
    "severity_counts = flood_events['severity'].value_counts()\n",
    "\n",
    "plt.figure(figsize=(8, 5))\n",
    "severity_counts.plot(kind='bar', color=['#dc143c', '#f59e0b', '#10b981'])\n",
    "plt.title('Flood Events by Severity', fontsize=13, weight='bold')\n",
    "plt.xlabel('Severity', fontsize=11)\n",
    "plt.ylabel('Count', fontsize=11)\n",
    "plt.xticks(rotation=0)\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(severity_counts)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Seasonal Risk Profile"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate seasonal risks\n",
    "seasonal_risk = analytics.seasonal_risk_profile()\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "colors = ['#dc2626' if x > 60 else '#f59e0b' if x > 30 else '#10b981' \n",
    "          for x in seasonal_risk['risk_score']]\n",
    "plt.bar(seasonal_risk['season'], seasonal_risk['risk_score'], color=colors)\n",
    "plt.title('Seasonal Flood Risk Scores', fontsize=13, weight='bold')\n",
    "plt.xlabel('Season', fontsize=11)\n",
    "plt.ylabel('Risk Score', fontsize=11)\n",
    "plt.ylim(0, 100)\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nSeasonal Risk Analysis:\")\n",
    "print(seasonal_risk[['season', 'flood_count', 'risk_score']].to_string(index=False))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Year-over-Year Trends"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare years\n",
    "yoy = analytics.year_over_year_comparison()\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "# Peak discharge by year\n",
    "ax1.bar(yoy['year'], yoy['peak_discharge'], color='#dc143c')\n",
    "ax1.set_title('Annual Peak Discharge', fontsize=12, weight='bold')\n",
    "ax1.set_xlabel('Year')\n",
    "ax1.set_ylabel('Discharge (m³/s)')\n",
    "ax1.grid(axis='y', alpha=0.3)\n",
    "\n",
    "# Flood days by year\n",
    "ax2.bar(yoy['year'], yoy['flood_days'], color='#003893')\n",
    "ax2.set_title('Annual Flood Days', fontsize=12, weight='bold')\n",
    "ax2.set_xlabel('Year')\n",
    "ax2.set_ylabel('Days')\n",
    "ax2.grid(axis='y', alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ Key Insights\n",
    "\n",
    "- Flood detection algorithm identifies events with 6+ hour duration\n",
    "- Events classified by severity (Minor, Moderate, Major)\n",
    "- Monsoon season shows highest risk as expected\n",
    "- Year-over-year analysis reveals inter-annual variability"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}